<a href="https://colab.research.google.com/github/leekate/college_project_DeepLearning/blob/main/FaceDetection_ResnetNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [ ]:
import numpy as np
x_train = np.load('./Train_np/X_train.npy')
y_train = np.load('./Train_np/y_train.npy')


In [ ]:
x_train.shape

(197012, 218, 178)

In [ ]:
y_train.shape

(197012, 136)

# 정규화

In [ ]:
#normalization
x_train=(x_train.astype(np.float32))/x_train.max()


In [ ]:
x_train=np.expand_dims(x_train,-1)

In [ ]:
x_train.shape

(197012, 218, 178, 1)

# 데이터 split

In [ ]:
#importing all neccessary packages
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten



In [ ]:
# 데이터(X)만 넣었을 경우
#x_train, x_test = train_test_split(x, test_size=0.2, random_state=123)


# 데이터(X)와 레이블(Y)을 넣었을 경우
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=321)


x_test=x_test.reshape(1332,1*256*256)
transformer.fit(x_test)

#MinMaxScaler 모델에 x_train_df 데이터 적용 (최소값, 최대값 계산)

x_test = transformer.transform(x_test)
print(x_test)

In [ ]:
#from tensorflow.python.keras.applications import ResNet50

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

In [ ]:
#Resnet디폴트 인풋 사이즈는 224x224
#근데 우리는 256,256으로 가자
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

save_dir="./save_model"
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
    
checkpointer=ModelCheckpoint(os.path.join(save_dir,'{epoch:0.3d}_valloss{val_loss:0.4f}.hdf5'),
                             verbose=1,
                             save_best_only=True)
es=EarlyStopping(monitor='val_loss',mode='min', baseline=0.1)


class createResnetModel:
    '''
    to create Resnet model
    using built-in functions of keras such as
    conv2D,
    maxpooling2D,
    flatten,
    dropout
    '''

    def __init__(self,input_size,output_size,epochs,batch_size):
        
        self.input_size = input_shape
        self.output_size = output_size
        self.epochs = epochs
        self.batch_size = batch_size
        
        self.createmodel()


    def createmodel(self):
        self.model = Sequential()

#         self.model.add(ResNet50(include_top=False, 
#                                 pooling='avg',
#                                weights=None))
        #model.add(ResNet50(include_top=False, 
                            #pooling='avg', 
                            #weights=resnet_weights_path))
        #weights: None (임의의 초기값 설정) 
        #혹은 'imagenet' (ImageNet에 대한 선행 학습) 중 하나.
        
        self.model.add(ResNet50(include_top=True, 
                                pooling='avg',
                                input_shape=(218,178,1),
                               weights=None))
        
        ################################DROPOUT 0.1로 10TIMES####################################
        self.model.add(Flatten())
        self.model.add(BatchNormalization())
        self.model.add(Dense(2048, activation='relu'))
        self.model.add(Dropout(0.3))
        
        self.model.add(BatchNormalization())
        self.model.add(Dense(2048, activation='relu'))
        self.model.add(Dropout(0.3))
        
        self.model.add(BatchNormalization())
        self.model.add(Dense(2048, activation='relu'))
        self.model.add(Dropout(0.3))
        
        self.model.add(BatchNormalization())
        self.model.add(Dense(1024, activation='relu'))
        self.model.add(Dropout(0.3))
        
        self.model.add(BatchNormalization())
        self.model.add(Dense(1024, activation='relu'))
        self.model.add(Dropout(0.3))

        self.model.add(BatchNormalization())
        self.model.add(Dense(1024, activation='relu'))
        self.model.add(Dropout(0.3))

        self.model.add(BatchNormalization())
        self.model.add(Dense(136, activation='softmax'))

        self.model.layers[0].trainable = False


    
    def summary(self):
        
        return self.model.summary()
    
    def compile(self,x_train, x_test, y_train, y_test,save= True):
        
        # Optimiser: the Optimiser  used to reduce the cost calculated by categorical_crossentropy
        # Loss: the loss function used to calculate the error
        # Metrics: the metrics used to represent the efficiency of the model
        # lr : learing rates
        
        adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
        
        self.model.compile(optimizer = adam,loss = 'mean_squared_error', metrics = ['acc'])
        self.model.fit(x = x_train, y= y_train,
                       batch_size= 64, 
                       verbose=1, 
                       epochs=100,
                       validation_data= (x_test,y_test),
                      callbacks=[checkpointer,es])
        if save:
            self.save()

    def save(self):
        
        # creates a HDF5 fil 'my_model.h5'
        self.model.save('my_Resnetmodel_10000.h5')  
        print("saved sucessfully")
        
    def save_history(self):
        
        # for visualizing losses and accuracy
        # History.history attribute is a record of training loss values
        # metrics values at successive epochs 
        # as well as validation loss values and validation metrics values
        train_loss=self.model.history['loss']
        val_loss=self.model.history['val_loss']
        train_acc=self.model.history['acc']
        val_acc=self.model.history['val_acc']
        xc=range(self.epochs)
        model_history = [
                train_loss,
                val_loss,
                train_acc,
                val_acc,
                xc]
        
        np.save('model_Resnet_history.npy',model_history)

In [ ]:

####################################################################################################################################
#pre-processing all data   
#원래:
#x_train = np.array(x_train).reshape(-1, 256,256,1).astype('float32')

#converting int datasets into float datasets 
# for easy processing by CPU/GPU
#x_train /= 255
x= np.median(y_train)
y_train = (y_train - x)/ x

# train_data /= 255
# x= np.median(ytrain)
# ytrain = (ytrain - x)/ x

#saving rows,col,dim of datasets to input_shape variable
#to give input size /shape to CNN model 
input_shape = x_train.shape[1:]

In [ ]:
x_train.shape

(157609, 218, 178, 1)

In [ ]:
x_test.shape

(39403, 218, 178, 1)

In [ ]:
#pre-processing all data   
#원래:
#x_test = np.array(x_test).reshape(-1, 256,256,1).astype('float32')

#converting int datasets into float datasets 
# for easy processing by CPU/GPU
#x_test /= 255
x= np.median(y_test)
y_test = (y_test - x)/ x

# train_data /= 255
# x= np.median(ytrain)
# ytrain = (ytrain - x)/ x

#saving rows,col,dim of datasets to input_shape variable
#to give input size /shape to CNN model 
input_shape = x_test.shape[1:]

In [ ]:
x_test.shape

(39403, 218, 178, 1)

In [ ]:

#model = createResnetModel((256,256),(256,256),700,64)
model = createResnetModel((218, 178),(136, ),700,64)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1000)              25630440  
_________________________________________________________________
flatten (Flatten)            (None, 1000)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1000)              4000      
_________________________________________________________________
dense (Dense)                (None, 2048)              2050048   
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4

In [ ]:
model.compile(x_train, x_test, y_train, y_test)



Train on 157609 samples, validate on 39403 samples
Epoch 1/100
157568/157609 [============================>.] - ETA: 0s - loss: 0.0873 - acc: 0.9117

ValueError: Precision not allowed in integer format specifier

In [ ]:
# model.save()
# model.save_history()